In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("data.csv",encoding='utf-8')

## 상관관계 분석

In [6]:
new_date=[]
for i in df.mea_ddhr:
    new_date.append(i[:-9])

In [7]:
df['new_d']=new_date

In [8]:
new_df=df.merge(pd.read_csv("train/plant11_train.csv").iloc[:,:6])

In [9]:
temp_gap=[]
for i in set(new_date):
    d_list=list(new_df.tem_in_loc.loc[new_df.new_d==i])
    x=max(d_list)
    n=min(d_list)
    temp_gap+=[x-n]*len(d_list)
    

In [10]:
tc_gap=[]
for i in range(len(df)):
    tc_gap.append(new_df.tem_in_loc[i]-new_df.tem_coil_loc[i])

In [11]:
new_df['tc_gap']=tc_gap
new_df['t_gap']=temp_gap

In [12]:
co=new_df.corr()

In [13]:
co[co.hum_in_loc_24hr>0.05].index

Index(['cond_24hr', 'tem_in_loc_24hr', 'hum_in_loc_24hr', 'tem_coil_loc_24hr',
       'month', 'week_25', 'week_26', 'week_27', 'week_28', 'week_29',
       'week_30', 'week_31', 'week_32', 'week_34', 'week_35', 'week_37',
       'week_38', 'week_45', 'dj_aws_tem', 'dj_aws_hum', 'sp_aws_tem',
       'tem_24hr', 'hum_24hr', 'rain_p_24hr', 'sky_4_24hr', 'rain_t_1_24hr',
       'tem_in_loc', 'hum_in_loc', 'tem_coil_loc', 'tem_out_loc',
       'hum_out_loc'],
      dtype='object')

In [14]:
co[co.hum_in_loc_24hr<-0.05].index

Index(['year', 'hour', 'week_1', 'week_3', 'week_4', 'week_5', 'week_6',
       'week_7', 'week_9', 'week_11', 'week_41', 'week_46', 'week_47',
       'week_49', 'week_50', 'week_52', 'dj_aws_wind_d', 'dj_aws_wind_s',
       'sp_aws_wind_d', 'sp_aws_wind_s', 'sp_aws_spot_pr', 'sp_aws_sea_pr',
       'sky_1_24hr', 'sky_2_24hr', 'rain_t_0_24hr', 'tc_gap'],
      dtype='object')

## correlation 이 높은 columns 기준으로 train

In [15]:
cond_col=['cond_24hr', 'hum_in_loc_24hr', 'week_20', 'week_48', 'week_49',
       'hum_24hr', 'rain_p_24hr', 'sky_4_24hr', 'rain_t_1_24hr', 'hum_in_loc',
       't_gap', 'tc_gap','tem_coil_loc_24hr', 'day', 'dj_aws_wind_d', 'sp_aws_wind_d',
       'sky_1_24hr', 'rain_t_0_24hr', 'tem_in_loc', 'tem_coil_loc']

In [19]:
data=new_df.loc[:,cond_col]

In [ ]:
new_df.to_csv("new_data.csv",encoding='utf-8',index=False)

In [22]:
X=data.drop(['cond_24hr','hum_in_loc_24hr','tem_coil_loc_24hr'],axis=1)
y=data.loc[:,'cond_24hr']

In [24]:
X_train=X.iloc[10000:]
X_test=X.iloc[:10000]
y_train=y.iloc[10000:]
y_test=y.iloc[:10000]

## Model_fit
- MLP, LGBM, XGB, RandomForest

In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [73]:
mlp_model=MLPClassifier()
lgb_model=LGBMClassifier()
xgb_model=XGBClassifier
rf_model=RandomForestClassifier()
model=[mlp_model,lgb_model,rf_model]

In [74]:
mlp_model=MLPClassifier()
lgb_model=LGBMClassifier()
xgb_model=XGBClassifier
rf_model=RandomForestClassifier()
model=[mlp_model,lgb_model,rf_model]
pred=[]
for i in model:
    i.fit(X_train.to_numpy(),y_train)
    pred.append(i.predict(X_test.to_numpy()))
    print(f1_score(pred[-1],y_test))

0.0
0.0
0.0


## Model_evaluation

In [75]:
lgb_model.fit(X_train.to_numpy(),y_train)
f1_score(lgb_model.predict(X_test.to_numpy()),y_test)

0.0

In [76]:
for i in pred:
    print(list(i).count(1))

6
37
0


In [37]:
pred[1]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [38]:
t=data.iloc[:10000]

In [39]:
t["mlp"]=pred[0]
t["lgb"]=pred[1]
t["rf"]=pred[2]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [42]:
t_l=t.loc[:,["cond_24hr","lgb"]]

In [43]:
len(t_l[t_l.cond_24hr==1]

,cond_24hr,lgb
84,1,0
112,1,0
233,1,0
249,1,0
600,1,0
601,1,0
602,1,0
2073,1,0
2551,1,0
2552,1,0


In [44]:
data.corr()

,cond_24hr,hum_in_loc_24hr,week_20,week_48,week_49,hum_24hr,rain_p_24hr,sky_4_24hr,rain_t_1_24hr,hum_in_loc,t_gap,tc_gap,tem_coil_loc_24hr,day,dj_aws_wind_d,sp_aws_wind_d,sky_1_24hr,rain_t_0_24hr,tem_in_loc,tem_coil_loc
cond_24hr,1.000000,0.167485,0.075534,0.172656,0.052714,0.067718,0.071886,0.083583,0.056686,0.035560,0.017851,0.118209,-0.042791,-0.066235,-0.032875,-0.043266,-0.034942,-0.052022,-0.040257,-0.054005
hum_in_loc_24hr,0.167485,1.000000,0.021317,-0.005843,-0.084735,0.325696,0.339738,0.271952,0.278904,0.525810,0.021464,-0.092082,0.444693,-0.036158,-0.241944,-0.237934,-0.198669,-0.252587,0.426462,0.434593
week_20,0.075534,0.021317,1.000000,-0.014707,-0.014781,-0.027803,0.019504,0.022400,0.045735,0.036844,-0.011377,0.048414,0.050001,0.019098,-0.013085,-0.025552,0.007781,-0.040656,0.051043,0.044972
week_48,0.172656,-0.005843,-0.014707,1.000000,-0.021908,0.008268,0.028133,0.026640,0.006926,-0.040411,0.096636,0.034201,-0.121983,0.068759,-0.046762,-0.015197,-0.068019,-0.002158,-0.116779,-0.120071
week_49,0.052714,-0.084735,-0.014781,-0.021908,1.000000,-0.102956,0.014967,-0.013883,-0.028837,-0.017970,0.052169,-0.012099,-0.188591,-0.162718,0.062403,0.050020,-0.052988,0.005555,-0.175623,-0.173044
hum_24hr,0.067718,0.325696,-0.027803,0.008268,-0.102956,1.000000,0.346429,0.298237,0.270081,0.123432,0.000258,0.327376,0.387237,0.001084,0.029948,-0.032451,-0.170403,-0.263876,0.420186,0.378642
rain_p_24hr,0.071886,0.339738,0.019504,0.028133,0.014967,0.346429,1.000000,0.780405,0.765019,0.097939,0.044565,0.018780,0.119431,-0.021796,-0.097105,-0.101810,-0.532508,-0.802104,0.124229,0.121193
sky_4_24hr,0.083583,0.271952,0.022400,0.026640,-0.013883,0.298237,0.780405,1.000000,0.580029,0.056448,0.048156,0.045379,0.053079,-0.010310,-0.092342,-0.086991,-0.250731,-0.616386,0.056016,0.050272
rain_t_1_24hr,0.056686,0.278904,0.045735,0.006926,-0.028837,0.270081,0.765019,0.580029,1.000000,0.123150,0.034066,0.018282,0.128570,0.024236,-0.081868,-0.080442,-0.155744,-0.945776,0.130847,0.127826
hum_in_loc,0.035560,0.525810,0.036844,-0.040411,-0.017970,0.123432,0.097939,0.056448,0.123150,1.000000,-0.008843,-0.023857,0.457217,-0.048904,-0.155320,-0.174649,-0.054408,-0.092923,0.443123,0.443059


In [46]:
data.iloc[:10000].loc[data.cond_24hr==1]

,cond_24hr,hum_in_loc_24hr,week_20,week_48,week_49,hum_24hr,rain_p_24hr,sky_4_24hr,rain_t_1_24hr,hum_in_loc,t_gap,tc_gap,tem_coil_loc_24hr,day,dj_aws_wind_d,sp_aws_wind_d,sky_1_24hr,rain_t_0_24hr,tem_in_loc,tem_coil_loc
84,1,78.00,0,0,0,53.000000,30.000000,1.0,0.0,46.00,4.43,2.00,14.00,11,130.2,124.4,0.0,1.0,16.00,14.00
112,1,80.00,0,0,0,74.500000,25.000000,1.0,0.0,50.00,4.79,4.00,14.00,15,256.5,285.1,0.0,1.0,16.00,12.00
233,1,80.00,0,0,0,48.000000,10.000000,0.0,0.0,64.00,6.26,3.00,18.00,30,149.6,198.3,0.0,1.0,21.00,18.00
249,1,80.00,0,0,0,83.000000,60.000000,1.0,1.0,64.00,6.26,3.00,18.00,2,284.6,80.7,0.0,0.0,21.00,18.00
600,1,78.00,0,0,0,84.500000,30.000000,1.0,0.0,68.00,4.42,0.00,19.00,15,167.0,214.2,0.0,1.0,25.00,25.00
601,1,76.00,0,0,0,74.000000,30.000000,1.0,0.0,70.00,4.42,0.00,19.00,15,187.0,155.1,0.0,1.0,24.00,24.00
602,1,74.00,0,0,0,70.000000,20.000000,0.0,0.0,72.00,4.42,-1.00,19.00,15,201.0,275.6,0.0,1.0,23.00,24.00
2073,1,78.79,0,0,0,49.166667,5.000000,0.0,0.0,47.37,2.53,-0.33,4.26,26,171.8,206.4,0.0,1.0,1.15,1.48
2551,1,68.01,0,0,0,90.000000,70.000000,1.0,1.0,31.35,1.49,3.52,6.05,15,151.3,273.2,0.0,0.0,9.15,5.63
2552,1,72.99,0,0,0,88.333333,70.000000,1.0,1.0,31.50,1.49,3.23,6.09,15,209.3,290.9,0.0,0.0,8.90,5.67


## train_test _split 다시하기
- train data가 불균형하게 존재해 test_set이 겨울철에만 한정되었음
- 따라서 2017년에 해당하는 칼럼들을 찾아 다시 train_test_split

In [57]:
l=list(new_df.mea_ddhr)
new_l=[]
for i in l:
    if i[:4]=='2017':
        new_l.append(l.index(i))

In [72]:
test_X=X.iloc[2039:9753]
test_y=y.iloc[2039:9753]
train_X=pd.concat([X.iloc[:2039],X.iloc[9753:]])
train_y=pd.concat([y.iloc[:2039],y.iloc[9753:]])


In [71]:
pd.concat([X.iloc[:2039],X.iloc[9753:]])


,week_20,week_48,week_49,hum_24hr,rain_p_24hr,sky_4_24hr,rain_t_1_24hr,hum_in_loc,t_gap,tc_gap,day,dj_aws_wind_d,sp_aws_wind_d,sky_1_24hr,rain_t_0_24hr,tem_in_loc,tem_coil_loc
0,0,0,0,42.0,10.0,0.0,0.0,24.00,5.62,5.00,1,207.1,216.9,0.0,1.0,16.00,11.00
1,0,0,0,42.0,10.0,0.0,0.0,28.00,5.62,4.00,1,131.5,154.0,0.0,1.0,14.00,10.00
2,0,0,0,37.0,20.0,0.0,0.0,33.00,5.62,3.00,1,161.5,200.6,0.0,1.0,13.00,10.00
3,0,0,0,40.0,20.0,0.0,0.0,33.00,5.62,3.00,1,307.3,161.4,0.0,1.0,13.00,10.00
4,0,0,0,59.0,20.0,0.0,0.0,28.00,5.62,6.00,1,23.2,15.4,0.0,1.0,16.00,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56995,0,0,0,70.0,20.0,0.0,0.0,47.37,4.03,0.41,29,330.3,294.1,0.0,1.0,12.99,12.58
56996,0,0,0,70.0,20.0,0.0,0.0,48.02,4.03,0.39,29,332.2,292.4,0.0,1.0,13.08,12.69
56997,0,0,0,70.0,20.0,0.0,0.0,48.63,4.03,0.51,29,311.6,287.3,0.0,1.0,13.17,12.66
56998,0,0,0,70.0,20.0,0.0,0.0,48.87,4.03,0.35,29,297.7,317.6,0.0,1.0,13.08,12.73


## Model_fit

In [77]:
mlp_model=MLPClassifier()
lgb_model=LGBMClassifier()
xgb_model=XGBClassifier
rf_model=RandomForestClassifier()
model=[mlp_model,lgb_model,rf_model]
pred=[]
for i in model:
    i.fit(train_X.to_numpy(),train_y)
    pred.append(i.predict(test_X.to_numpy()))
    print(f1_score(pred[-1],test_y))

0.0625
0.0
0.0


In [79]:
pd.crosstab(pred[0],test_y)

cond_24hr,0,1
row_0,,
0,7683,26
1,4,1


In [82]:
test=pd.read_csv("test/plant_test.csv",encoding='utf-8',index_col=0)

In [83]:
test.columns

Index(['plant_test.mea_ddhr', 'plant_test.plant', 'plant_test.loc',
       'plant_test.tem_in', 'plant_test.hum_in', 'plant_test.tem_coil',
       'plant_test.tem_out_loc1', 'plant_test.hum_out_loc1',
       'plant_test.x24h_tma', 'plant_test.x24h_cond_loc',
       'plant_test.x48h_tma', 'plant_test.x48h_cond_loc'],
      dtype='object')

In [84]:
test['plant_test.plant']

1       2
2       1
3       2
4       1
5       2
       ..
3535    1
3536    2
3537    2
3538    1
3539    1
Name: plant_test.plant, Length: 3539, dtype: int64

## 변경된 train_test set으로 학습후 제출

In [86]:
list1=[]
list2=[]
plant_list=list(test['plant_test.plant'])
for i in range(len(test)):
    if plant_list[i]==1:
        list1.append(1)
    elif plant_list[i]==2:
        list1.append(0)

In [89]:
test['plant_test.x48h_cond_loc']=list1

In [90]:
test

,plant_test.mea_ddhr,plant_test.plant,plant_test.loc,plant_test.tem_in,plant_test.hum_in,plant_test.tem_coil,plant_test.tem_out_loc1,plant_test.hum_out_loc1,plant_test.x24h_tma,plant_test.x24h_cond_loc,plant_test.x48h_tma,plant_test.x48h_cond_loc
1,2019-04-01 0:00,2,3,8.17,40.42,10.10,4.20,54.82,2019-04-02 0:00,0,2019-04-03 0:00,0
2,2019-04-01 4:30,1,2,10.03,48.81,10.79,6.09,59.34,2019-04-02 4:30,1,2019-04-03 4:30,1
3,2019-04-01 10:30,2,3,9.45,40.93,9.07,12.26,32.14,2019-04-02 10:30,0,2019-04-03 10:30,0
4,2019-04-01 11:00,1,1,12.32,34.13,10.86,12.32,28.27,2019-04-02 11:00,1,2019-04-03 11:00,1
5,2019-04-01 13:30,2,1,11.28,22.65,8.76,12.01,19.84,2019-04-02 13:30,0,2019-04-03 13:30,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3535,2020-03-29 3:00,1,2,10.73,33.36,13.30,6.28,46.37,2020-03-30 3:00,1,2020-03-31 3:00,1
3536,2020-03-29 5:00,2,3,9.15,38.86,11.70,1.39,74.33,2020-03-30 5:00,0,2020-03-31 5:00,0
3537,2020-03-29 7:00,2,1,7.10,44.87,9.03,4.26,58.06,2020-03-30 7:00,0,2020-03-31 7:00,0
3538,2020-03-29 11:30,1,1,13.54,33.49,12.20,12.90,33.67,2020-03-30 11:30,1,2020-03-31 11:30,1


In [93]:
test=pd.read_csv("203771.csv",encoding='utf-8')
test.columns

Index(['MEA_DDHR', 'PLANT', 'LOC', 'X24H_TMA', 'X24H_COND_LOC',
       'X24H_COND_LOC_PROB', 'X48H_TMA', 'X48H_COND_LOC',
       'X48H_COND_LOC_PROB'],
      dtype='object')

In [94]:
list1=[]
list2=[]
plant_list=list(test['PLANT'])
for i in range(len(test)):
    if plant_list[i]==1:
        list1.append(1)
    elif plant_list[i]==2:
        list1.append(0)

In [96]:
test['X24H_COND_LOC']=list1
test['X48H_COND_LOC']=list1

In [99]:
test.to_csv("203771_t.csv",encoding='utf-8',index=False)

In [ ]:
t